In [3]:
spark.stop()

25/05/22 13:36:28 INFO  SparkContext:60 SparkContext is stopping with exitCode 0.
25/05/22 13:36:28 INFO  SparkUI:60 Stopped Spark web UI at http://6af08edc54fe:4041
25/05/22 13:36:28 INFO  StandaloneSchedulerBackend:60 Shutting down all executors
25/05/22 13:36:28 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Asking each executor to shut down
25/05/22 13:36:28 INFO  MapOutputTrackerMasterEndpoint:60 MapOutputTrackerMasterEndpoint stopped!
25/05/22 13:36:28 INFO  MemoryStore:60 MemoryStore cleared
25/05/22 13:36:28 INFO  BlockManager:60 BlockManager stopped
25/05/22 13:36:28 INFO  BlockManagerMaster:60 BlockManagerMaster stopped
25/05/22 13:36:28 INFO  OutputCommitCoordinator$OutputCommitCoordinatorEndpoint:60 OutputCommitCoordinator stopped!
25/05/22 13:36:28 INFO  SparkContext:60 Successfully stopped SparkContext


In [4]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Streaming from Kafka") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config("spark.sql.shuffle.partitions", 4)
    .getOrCreate()
)

spark

25/05/22 13:36:29 INFO  SparkContext:60 Running Spark version 3.5.5
25/05/22 13:36:29 INFO  SparkContext:60 OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/05/22 13:36:29 INFO  SparkContext:60 Java version 17.0.14
Setting Spark log level to "INFO".
25/05/22 13:36:29 INFO  ResourceUtils:60 ==============================================================
25/05/22 13:36:29 INFO  ResourceUtils:60 No custom resources configured for spark.driver.
25/05/22 13:36:29 INFO  ResourceUtils:60 ==============================================================
25/05/22 13:36:29 INFO  SparkContext:60 Submitted application: Streaming from Kafka
25/05/22 13:36:29 INFO  ResourceProfile:60 Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 4, script: , vendor: , memory -> name: memory, amount: 2048, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/05/22 13:36:29 INFO  ResourcePro

In [2]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "broker-1:29092,broker-2:39092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)


Py4JJavaError: An error occurred while calling o59.load.
: java.lang.NoClassDefFoundError: org/apache/kafka/common/serialization/ByteArraySerializer
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.<init>(KafkaSourceProvider.scala:601)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.<clinit>(KafkaSourceProvider.scala)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:338)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:233)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:118)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:118)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:36)
	at org.apache.spark.sql.str

In [23]:
# View schema for raw kafka_df
kafka_df.printSchema()
#kafka_df.show()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [24]:
# Parse value from binay to string into kafka_json_df
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [25]:
# Schema of the Pyaload

from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType

json_schema = (
    StructType(
        [
            StructField('customerId', StringType(), True), 
            StructField('data',
                        StructType(
                            [
                                StructField('devices',
                                            ArrayType(
                                                StructType(
                                                    [ 
                                                        StructField('deviceId', StringType(), True), 
                                                        StructField('measure', StringType(), True), 
                                                        StructField('status', StringType(), True), 
                                                        StructField('temperature', LongType(), True)
                                                    ]
                                                ), True
                                            ), True
                                )
                            ]
                        ), True
            ), 
            StructField('eventId', StringType(), True), 
            StructField('eventOffset', LongType(), True), 
            StructField('eventPublisher', StringType(), True), 
            StructField('eventTime', StringType(), True)
        ]
    )
)

In [26]:
# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [27]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
#streaming_df.show(truncate=False)

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)



In [28]:
# Lets explode the data as devices contains list/array of device reading
from pyspark.sql.functions import explode

exploded_df = streaming_df.withColumn("data_devices", explode("data.devices"))


In [29]:
# Check the schema of the exploded_df, place a sample json file and change readStream to read 
exploded_df.printSchema()
#exploded_df.show(truncate=False)

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- data_devices: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- measure: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- temperature: long (nullable = true)



In [30]:
# Flatten the exploded df
from pyspark.sql.functions import col

flattened_df = (
    exploded_df
    .drop("data")
    .withColumn("deviceId", col("data_devices.deviceId"))
    .withColumn("measure", col("data_devices.measure"))
    .withColumn("status", col("data_devices.status"))
    .withColumn("temperature", col("data_devices.temperature"))
    .drop("data_devices")
)


In [31]:
# Check the schema of the flattened_df, place a sample json file and change readStream to read 
flattened_df.printSchema()
#flattened_df.show(truncate=False)

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: long (nullable = true)



In [ ]:
# Write the output to console sink to check the output

(flattened_df
 .writeStream
 .format("console")
 .outputMode("append")
 .option("checkpointLocation", "/opt/spark/spark-checkpoint")
 .start()
 .awaitTermination())

In [ ]:
# Check the data at the output location

